In [0]:
#for creation of labelled_data.txt
import re
import pandas as pd  #for preparing training data
from collections import Counter
import itertools
from keras.preprocessing.text import text_to_word_sequence

In [0]:
# tags from corpus
tagged_corpus = pd.read_csv("data/corpus_tags.tsv", sep="\t", names=["pmid", "start", "end", "word", "type", "umls_id"])
tagged_corpus = tagged_corpus.dropna()
tagged_corpus.head()

In [0]:
classes = []
with open("data/vocab_class.txt") as ftags:
    classes = [i.strip() for i in ftags.readlines()]
    ftags.close()
classes

In [0]:
# for each sentence the words and tags for each words is returned
def get_words_tags(text, pmid,start=0):
    words = []
    tags = []  
    typ = "O"
    df = tagged_corpus[(tagged_corpus["pmid"]==int(pmid)) & (tagged_corpus["end"] <= start + len(text)) & (tagged_corpus["start"] >= start)]
    indices = df.index.values
    text_split = text_to_word_sequence(text)
    for word in text_split:
        words.append(word)
        for i in indices:
            text2 = df.loc[i]["word"]
            text2_split = text_to_word_sequence(text2)
            #if word in text2_split and set(df.loc[i]["type"].split(',')).intersection(classes):
            if word in text2_split:
                typ = df.loc[i]["type"]
                break
            else:
                typ = "O"
        tags.append(typ)
    return words,tags

In [0]:
fr = open("data/corpus_pubtator.txt")
#fr = open("para.txt")
fw = open("data/labelled_data.txt", "w")

title_regex = re.compile("^\d+\|t\|")
abstract_regex = re.compile("^\d+\|a\|") 

In [0]:
#opens the file and sends the title and abstract line by line to get_words_tags function
abstract_count = 0
for i in fr.readlines():
    line = i.strip()
    title_search = title_regex.search(line)
    abstract_search = abstract_regex.search(line)
    if title_search:
        title_line = title_regex.sub('',line)
    elif abstract_search:
        abstract = abstract_regex.sub('',line)
        pmid = int(abstract_search.group(0)[:-3])
        words,tags = get_words_tags(title_line,pmid,0)
        fw.write("\t".join(words)+"\n")
        fw.write("\t".join(tags)+"\n")
        title_size = len(title_line) + 1                     # 1 for \n        
        for sentence in re.split("\.", abstract):
            if len(sentence) == 0:
                continue
            _size = abstract.index(sentence) + title_size
            words, tags = get_words_tags(sentence, pmid, _size)
            fw.write("\t".join(words)+"\n")
            fw.write("\t".join(tags)+"\n") 
        abstract_count += 1
        if abstract_count%10 == 0:
            print("PMID {} written to file. Abstract Count: {}".format(pmid, abstract_count))
fw.close()
fr.close()